 # Zernike Test
## Setup the required functions and input images

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Feb  1 20:06:06 2019

@author: vik748
"""
import numpy as np
import cv2
import sys
from matplotlib import pyplot as plt
%matplotlib inline
from zernike import MultiHarrisZernike
from vslam_helper import *

def draw_keypoints(vis_orig, keypoints, color = (255, 255, 0), thick = 1):
    vis = vis_orig.copy()
    for kp in keypoints:
        x, y = kp.pt
        cv2.circle(vis, (int(x), int(y)), int(vis.shape[1]/200), color, thickness=thick)
    return vis

def draw_markers(vis_orig, keypoints, color = (0, 0, 255)):
    vis = vis_orig.copy()
    for kp in keypoints:
        x, y = kp.pt
        cv2.drawMarker(vis, (int(x), int(y)), color,  markerSize=30, markerType = cv2.MARKER_CROSS, thickness=3)
    return vis

def draw_marker_pts(vis_orig, pts, color = (0, 0, 255)):
    vis = vis_orig.copy()
    for row in pts[:,0,:]:
        cv2.drawMarker(vis, (int(row[0]), int(row[1])), color,  markerSize=30, markerType = cv2.MARKER_CROSS, thickness=3)
    return vis

def draw_arrows(vis_orig, points1, points2, color = (0, 255, 0), thick = 2):
    vis = cv2.cvtColor(vis_orig,cv2.COLOR_GRAY2RGB)
    #rad = int(vis.shape[1]/200)
    for p1,p2 in zip(points1,points2):
        cv2.arrowedLine(vis, (int(p1[0]),int(p1[1])), (int(p2[0]),int(p2[1])), color=(0,255,0), thickness=thick)
    return vis

def draw_feature_tracks(img_left,kp1,img_right,kp2, matches, mask, display_invalid=False, color=(0, 255, 0)):
    '''
    This function extracts takes a 2 images, set of keypoints and a mask of valid
    (mask as a ndarray) keypoints and plots the valid ones in green and invalid in red.
    The mask should be the same length as matches
    '''
    if mask is None: bool_mask = np.ones((len(matches)), dtype=bool)
    else: bool_mask = mask.astype(bool)
    valid_right_matches = np.array([kp2[mat.trainIdx].pt for is_match, mat in zip(bool_mask, matches) if is_match])
    valid_left_matches = np.array([kp1[mat.queryIdx].pt for is_match, mat in zip(bool_mask, matches) if is_match])
    #img_right_out = draw_points(img_right, valid_right_matches)
    img_right_out = draw_arrows(img_right, valid_left_matches, valid_right_matches)
    return img_right_out

def displayMatches(img_left,kp1,img_right,kp2, matches, mask, display_invalid, in_image=None, color=(0, 255, 0)):
    '''
    This function extracts takes a 2 images, set of keypoints and a mask of valid
    (mask as a ndarray) keypoints and plots the valid ones in green and invalid in red.
    The mask should be the same length as matches
    '''
    bool_mask = mask.astype(bool)
    if in_image is None: mode_flag=0
    else: mode_flag =1
    img_valid = cv2.drawMatches(img_left,kp1,img_right,kp2,matches, in_image, 
                                matchColor=color, 
                                matchesMask=bool_mask.ravel().tolist(), flags=mode_flag)
    
    if display_invalid:
        img_valid = cv2.drawMatches(img_left,kp1,img_right,kp2,matches, img_valid, 
                                  matchColor=(255, 0, 0), 
                                  matchesMask=np.invert(bool_mask).ravel().tolist(), 
                                  flags=1)
    return img_valid


K = np.array([[718.856, 0.0, 607.1928],
              [0.0, 718.856, 185.2157],
              [0.0, 0.0, 1.0]])

D =  np.array([0.0, 0.0, 0.0, 0.0, 0.0])

if sys.platform == 'darwin':
    path = '/Users/vik748/Google Drive/'
else:
    path = '/home/vik748/'
img1 = cv2.imread(path+'data/kitti/00/image_0/000001.png',0) # iscolor = CV_LOAD_IMAGE_GRAYSCALE
img2 = cv2.imread(path+'data/kitti/00/image_0/000003.png',0) # iscolor = CV_LOAD_IMAGE_GRAYSCALE

#gr1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
#gr2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
gr1 = img1
gr2 = img2

# create a mask image filled with 1s, the size of original image
#m1 = cv2.imread(path+'data/time_lapse_5_cervino_800x600_masks_out/G0057821_mask.png',cv2.IMREAD_GRAYSCALE)
#m2 = cv2.imread(path+'data/time_lapse_5_cervino_800x600_masks_out/G0057826_mask.png',cv2.IMREAD_GRAYSCALE)

## Feature Detection

In [ ]:

detector = cv2.ORB_create(nfeatures=600, scaleFactor=2.0, nlevels=4, edgeThreshold=31, 
                           firstLevel=0, WTA_K=2, scoreType=cv2.ORB_HARRIS_SCORE,  
                           patchSize=31, fastThreshold=15)
'''
detector = MultiHarrisZernike(Nfeats=600)
'''
# find the keypoints and descriptors with ORB
kp1,des1 = detector.detectAndCompute(gr1,None)
kp2,des2 = detector.detectAndCompute(gr2,None)
print("Detected kps1: ",len(kp1)," kp2: ",len(kp2))

kp1_top = sorted(kp1, key = lambda x:x.response)[-25:]
kp2_top = sorted(kp2, key = lambda x:x.response)[-25:]

img_out1 = cv2.drawKeypoints(gr1, kp1_top, gr1,color=[255,255,0],
                             flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
img_out2 = cv2.drawKeypoints(gr2, kp2_top, gr2,color=[255,255,0],
                             flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

fig, [ax1, ax2] = plt.subplots(1,2,dpi=200)
fig.suptitle('800x600 Yellow circle defult setting patchSize=31')
ax1.axis("off")
ax1.imshow(img_out1)
ax2.axis("off")
ax2.imshow(img_out2)
fig.subplots_adjust(0,0,1,1,0.0,0.0)
plt.show()

## Feature matching

In [ ]:
'''
FLANN_INDEX_KDTREE = 0
matcher = cv2.FlannBasedMatcher(dict(algorithm = FLANN_INDEX_KDTREE, trees = 4), dict(checks=32) )
'''
matcher = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)

matches_12 = knn_match_and_lowe_ratio_filter(matcher, des1, des2, threshold=0.9)
track_output = track_keypoints(kp1, des1, kp2, des2, matches_12)
kp1_matched, des1_matched, kp2_matched, des2_matched, kp2_cand, des2_cand = track_output
print("Found matches: ",len(matches_12))

kp1_match_12 = np.expand_dims(np.array([o.pt for o in kp1_matched],dtype=np.float32),1)
kp2_match_12 = np.expand_dims(np.array([o.pt for o in kp2_matched],dtype=np.float32),1)

kp1_match_12_ud = cv2.undistortPoints(kp1_match_12,K,D)
kp2_match_12_ud = cv2.undistortPoints(kp2_match_12,K,D)

E_12, mask_e_12 = cv2.findEssentialMat(kp1_match_12_ud, kp2_match_12_ud, focal=1.0, pp=(0., 0.), 
                                       method=cv2.RANSAC, prob=0.9999, threshold=0.001)

print("After essential: ", np.sum(mask_e_12))

points, R_21, t_21, mask_RP_12 = cv2.recoverPose(E_12, kp1_match_12_ud, kp2_match_12_ud,mask=mask_e_12)
print("After RP: ", np.sum(mask_RP_12))
img_valid = draw_feature_tracks(gr1,kp1,gr2,kp2, matches_12, mask_RP_12, display_invalid=True, color=(0, 255, 0))
#img_valid = displayMatches(gr1,kp1,gr2,kp2, matches_12, mask_e_12, display_invalid=False, color=(0, 255, 0))

fig, ax= plt.subplots(dpi=200)
plt.title('800x600 Zernike Matches after using findEssential mask')
plt.axis("off")
plt.imshow(img_valid)
plt.show()

In [ ]:
#img_valid = displayMatches(gr1,kp1,gr2,kp2, matches, None, display_invalid=False, color=(0, 255, 0))
img_valid = cv2.drawMatches(gr1,kp1,gr2,kp2,matches, None, flags=0)
fig, ax= plt.subplots(dpi=200)
plt.title('800x600 Yellow circle defult setting patchSize=31')
plt.axis("off")
plt.imshow(img_valid)
plt.show()

In [ ]:
p = kp1[0]
import copy
kp_copy = copy.deepcopy(kp1)
p_copy = copy.deepcopy(p)

In [ ]:
kp_copy[0].pt